In [ ]:
import pandas as pd
import io
import math
import numpy

In [ ]:
dataFrame = pd.read_csv("Mortality_09_UP.data",nrows=25175)

In [ ]:
dataFrame

In [ ]:
columns = ["age","sex", "highest_qualification", "rural", "disability_status", "is_water_filter", "chew", "smoke", "alcohol","treatment_source"]
death = dataFrame[columns].copy()

for column in columns:
    death[column].fillna(death[column].mode()[0], inplace=True)

In [ ]:
death

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

print(death.isnull().sum())

le = LabelEncoder()
death['treatment_source'] = le.fit_transform(death['treatment_source'])

death = pd.get_dummies(death, columns=['sex', 'rural', 'disability_status', 'is_water_filter', 'chew', 'smoke', 'alcohol'])

X = death.drop(['age'], axis=1)
y = death['age']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score

rf = RandomForestRegressor(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)

y_pred = rf.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
rmse = math.sqrt(mse)
r2 = r2_score(y_test, y_pred)

print('MSE:', mse)
print('RMSE:', rmse)
print('R-squared:', r2)


In [ ]:
from sklearn.feature_selection import RFE

rf = RandomForestRegressor(n_estimators=100, random_state=42)

rfe = RFE(estimator=rf, n_features_to_select=5, step=1)
rfe.fit(X_train, y_train)

print('Top 5 Features:', X_train.columns[rfe.support_])


In [ ]:
X_train_rfe = rfe.transform(X_train)
X_test_rfe = rfe.transform(X_test)

rf.fit(X_train_rfe, y_train)

y_pred = rf.predict(X_test_rfe)
mse = mean_squared_error(y_test, y_pred)
rmse = math.sqrt(mse)
r2 = r2_score(y_test, y_pred)

print('MSE:', mse)
print('RMSE:', rmse)
print('R-squared:', r2)


In [ ]:
x = death.drop(X_train.columns[rfe.support_] , axis=1)
y = death['age']

In [ ]:
x.dtypes

In [ ]:
# columns_new = ['highest_qualification', 'treatment_source', 'disability_status_0.0','chew_5.0', 'alcohol_4.0']
# death_new = death[columns_new].copy()

In [ ]:
# death_new.dtypes

In [ ]:
# death_new

In [ ]:
y

In [ ]:
#train-test split
from sklearn.model_selection import train_test_split

X_train, X_test,y_train, y_test = train_test_split(x,y ,random_state=104,test_size=0.25,shuffle=True)

In [ ]:
import tensorflow as tf


In [ ]:
import tensorflow as tf

bias=tf.keras.initializers.HeNormal()

model = tf.keras.Sequential([
  tf.keras.layers.Dense(10, kernel_initializer="he_normal", bias_initializer=bias, activation='sigmoid', input_shape=(30,)),
  tf.keras.layers.Dense(20, kernel_initializer="he_normal", bias_initializer=bias, activation='relu'),
  tf.keras.layers.Dense(6, kernel_initializer="he_normal", bias_initializer=bias, activation='swish', input_shape=(30,)),
  tf.keras.layers.Dense(6, kernel_initializer="he_normal", bias_initializer=bias, activation='sigmoid', input_shape=(30,)),
  tf.keras.layers.Dense(6, kernel_initializer="he_normal", bias_initializer=bias, activation='sigmoid', input_shape=(30,)),
])


In [ ]:
model.summary()

In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adam(0.1),loss=tf.keras.losses.MeanAbsoluteError(),metrics=[tf.keras.metrics.RootMeanSquaredError() , 'accuracy'])

In [ ]:
learning=model.fit(X_train,y_train,epochs=20,batch_size=16,verbose=True)

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
def plot(learning):
  plt.plot(learning.history['loss'])
  plt.xlabel("epochs")
  plt.ylabel("Cost")

  plt.show()


In [ ]:
plot(learning)

In [ ]:
model.weights

In [ ]:
import joblib

joblib.dump(model,"model2.pkl")

In [ ]:
#loading
p_model=joblib.load("model2.pkl")

In [ ]:
p_model.summary()

In [ ]:
p_model.predict(X_test)

In [ ]:
X_test.head()

In [ ]:
jsonfile=model.to_json()
with open ("model.json","w") as f:
  f.write(jsonfile)

model.save_weights("model.h5")

In [ ]:
y_test